<a href="https://colab.research.google.com/github/akash-kaul/ecosys/blob/master/FlaskApi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Choose Model Type

In [ ]:
#@title
from ipywidgets import widgets, Layout
from IPython.display import display, Image
from ipywidgets.widgets import VBox, HBox
import threading
from IPython.utils import io
import os

if not os.path.exists('/content/Loading_GIFs'):
    with io.capture_output() as captured:
        !git clone https://github.com/akash-kaul/Loading_GIFs.git
## CREATE DROPDOWN FOR NLP MODELS ##
out = widgets.Output()
model = widgets.Dropdown(
    options=["Spacy - core", "SciSpacy - core", "Scispacy - craft", "Scispacy - jnlpba", "Scispacy - bc5cdr", "Scispacy - bionlp", "Sentiment - Vader", "Sentiment - TextBlob"],
    description='Choose a NLP Model',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=Layout(margin='0px 0px 20px 0px')
)
select = widgets.Button(
    description='Launch API',
    disabled=False,
    button_style='danger',
    tooltip='Click to launch API',
    icon='check',
    layout=Layout(align_items='stretch', width='100%')
)

box_layout = Layout(display='flex',
                flex_flow='column',
                align_items='flex-end',
                width='21%',
                )
def getPackages(ModelType):
    nlp = None
    analyzer = None
    textblobcheck = False
    with io.capture_output() as captured:
        !pip install flask-ngrok flask_restful
    if ModelType == 'Spacy - core':
        with io.capture_output() as captured:
            !pip install spacy
            !python -m spacy download en_core_web_md
        import spacy
        import en_core_web_md
        nlp = en_core_web_md.load()
        print('lsakfjd')
    elif ModelType == 'SciSpacy - core':
        with io.capture_output() as captured:
            !pip install scispacy
            !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_core_sci_md-0.2.5.tar.gz
        import scispacy
        import en_core_sci_md
        nlp = en_core_sci_md.load()
    elif ModelType == "Scispacy - craft":
        with io.capture_output() as captured:
            !pip install scispacy
            !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_ner_craft_md-0.2.5.tar.gz
        import scispacy
        import en_ner_craft_md
        nlp = en_ner_craft_md.load()
    elif ModelType == "Scispacy - bc5cdr":
        with io.capture_output() as captured:
            !pip install scispacy
            !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_ner_bc5cdr_md-0.2.5.tar.gz
        import scispacy
        import en_ner_bc5cdr_md
        nlp = en_ner_bc5cdr_md.load()
    elif ModelType == "Scispacy - jnlpba":
        with io.capture_output() as captured:
            !pip install scispacy
            !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_ner_jnlpba_md-0.2.5.tar.gz
        import scispacy
        import en_ner_jnlpba_md
        nlp = en_ner_jnlpba_md.load()
    elif ModelType == "Scispacy - bionlp":
        with io.capture_output() as captured:
            !pip install scispacy
            !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_ner_bionlp13cg_md-0.2.5.tar.gz
        import scispacy
        import en_ner_bionlp13cg_md
        nlp = en_ner_bionlp13cg_md.load()
    elif ModelType == 'Sentiment - Vader':
        with io.capture_output() as captured:
            !pip install flask_restful vaderSentiment
        from vaderSentiment import vaderSentiment
        analyzer = vaderSentiment.SentimentIntensityAnalyzer()
    elif ModelType == "Sentiment - TextBlob":
        with io.capture_output() as captured:
            !pip install textblob
        from textblob import TextBlob
        textblobcheck=True


    return nlp, analyzer, textblobcheck

def launchAPI(b):
    select.button_style='warning'
    select.description='Launching API'

    nlp, analyzer, check = getPackages(model.value)

    out.clear_output()
    select.description='API Launched'
    select.button_style='success'
    with out:
        display(Image('/content/Loading_GIFs/unknown.png'))
    
    from flask import Flask, jsonify
    from flask_restful import reqparse, abort, Api, Resource
    from flask_ngrok import run_with_ngrok

    app = Flask(__name__)
    run_with_ngrok(app)
    api = Api(app)

    parser = reqparse.RequestParser()
    parser.add_argument('query')

    class GetEntities(Resource):
        def get(self):
            args = parser.parse_args()
            user_query = args['query']

            if check:
                from textblob import TextBlob
                text = TextBlob(user_query)
                return jsonify(text.sentiment)
            elif nlp is None:
                polarity = analyzer.polarity_scores(str(user_query))
                return jsonify(polarity)
            else:
                doc = nlp(user_query)
                output = {}
                for ent in doc.ents:
                    output[ent.text] = ent.label_
                return jsonify(output)
            # if nlp is None:
            #     if analyzer is None:
            #         from textblob import TextBlob
            #         text = TextBlob(user_query)
            #         return jsonify(text.sentiment)
            #     else:
            #         polarity = analyzer.polarity_scores(str(user_query))
            #         return jsonify(polarity)
            # else:
            #     doc = nlp(user_query)
            #     output = {}
            #     for ent in doc.ents:
            #         output[ent.text] = ent.label_
            #     return jsonify(output)

    api.add_resource(GetEntities, '/')

    if __name__ == '__main__':
        app.run()

def playDino(b):
    out.clear_output()
    with out:
        display(Image('/content/Loading_GIFs/dino.gif', format='png'))

select.on_click(playDino)
select.on_click(launchAPI)

box = VBox([model, out, select], layout=box_layout)
display(box)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9980d957d8bf.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [19/Aug/2020 18:51:49] "GET /?query=my%20doctor%20sucks%20I%20want%20a%20new%20one HTTP/1.1" 200 -
